In [1]:
import os

In [2]:
%pwd

'e:\\chicken_disease_classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\chicken_disease_classification'

In [5]:
#for entity
from  dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:

    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int
    
    



In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,created_directories


In [7]:
# for configuration

class ConfigurationManager:
    def __init__(
        self, config_filepth=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH,
    ):
        self.config = read_yaml(config_filepth)
        self.params = read_yaml(params_filepath)

        created_directories([self.config.artifacts_root])
        

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model

        created_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES,
            
        )
        return prepare_base_model_config

In [8]:
#for component as prepare_base_model
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model  # Add this import


[2023-12-31 17:00:40,247:WARNING:module_wrapper:From e:\chicken_disease_classification\chicken_ven\lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.
]


In [9]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
        self.model=None
        self.full_model=None
        self.base_model=None


    def get_base_model(self):
        if self.config is not None:
            self.model = tf.keras.applications.vgg16.VGG16(
                input_shape=self.config.params_image_size,
                weights=self.config.params_weights,
                include_top=self.config.params_include_top,
                
            )

        # saving the model

        self.save_model(path=self.config.base_model_path, model=self.model)

    @staticmethod
    def prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        # checkig the layer

        if freeze_all:
            for layer in model.layers:
                model.trainable = False

        elif (freeze_till is not None) and (freeze_till) > 0:
            for layer in model.layers:
                model.trainable = True

        # flatten layer

        flatten_in = tf.keras.layers.Flatten()(model.output)
        # last layer
        predicton = tf.keras.layers.Dense(units=classes, activation="softmax")(
            flatten_in
        )

        full_model = Model(model.input, outputs=predicton)
        # compiling the model
        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"],
        )

        full_model.summary()
        return full_model
        
        
    def update_base_model(self):
        if self.model is not None:
            print("Model in update_base_model:", self.model)

            self.full_model=self.prepare_full_model(
                model=self.model,
                classes=self.config.params_classes,
                freeze_all=True,
                freeze_till=None,
                learning_rate=self.config.params_learning_rate
            )
        
        self.save_model(path=self.config.updated_base_model_path,model=self.full_model)
        
    @staticmethod
    def save_model(path:Path,model:tf.keras.Model):
        model.save(path)
        

In [10]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2023-12-31 17:00:41,175:INFO:common:yaml file:config\config.yaml loadded successfully]
[2023-12-31 17:00:41,179:INFO:common:yaml file:params.yaml loadded successfully]
[2023-12-31 17:00:41,181:INFO:common:Created directory as:{path}]
[2023-12-31 17:00:41,182:INFO:common:Created directory as:{path}]
[2023-12-31 17:00:41,363:WARNING:module_wrapper:From e:\chicken_disease_classification\chicken_ven\lib\site-packages\keras\src\backend.py:1398: The name tf.executing_eagerly_outside_functions is deprecated. Please use tf.compat.v1.executing_eagerly_outside_functions instead.
]
[2023-12-31 17:00:41,504:WARNING:module_wrapper:From e:\chicken_disease_classification\chicken_ven\lib\site-packages\keras\src\layers\pooling\max_pooling2d.py:161: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.
]
[2023-12-31 17:00:41,916:WARNING:saving_utils:Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or ev

e:\chicken_disease_classification\chicken_ven\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       295168    
                                                                 
 block3_conv2 (Conv2D)       (None, 56, 56, 256)       590080    
                                                                 
 block3_conv3 (Conv2D)       (None, 56, 56, 256)       590080    
                                                                 
 block3_pool (MaxPooling2D)  (None, 28, 28, 256)       0         
                                                                 
 block4_conv1 (Conv2D)       (None, 28, 28, 512)       1180160   
          